TOTO予測
===

# データ収集

[公式記録](https://data.j-league.or.jp/)からデータを収集する。

APIとかで取得できるようにはなっていないので、HTMLをパースして加工しやすいような形式にすることを考える。

`https://data.j-league.or.jp/SFMS01/search?competition_years=<年度>&competition_frame_ids=<カテゴリ>`

In [1]:
import requests